In [24]:
import networkx as nx
import pandas as pd
import numpy as np
import pulp
df = pd.read_csv("dataset/finished_dataset.csv")

# Load the graph from the Pickle file
graph_from_routes = nx.read_gpickle("graph_from_routes_wd.pkl")


In [25]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_date,location_pickup,location_dropoff,pickup_graph_node,dropoff_graph_node
0,0,0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016-03-14,5,22,42435716,42447076
1,1,42,id2129090,1,2016-03-14 14:05:39,2016-03-14 14:28:05,1,-73.975090,40.758766,-73.953201,40.765068,N,1346,2016-03-14,12,11,42445926,42429694
2,2,58,id0256505,1,2016-03-14 15:04:38,2016-03-14 15:16:13,1,-73.994484,40.745087,-73.998993,40.722710,N,695,2016-03-14,30,4,42437890,42458267
3,3,103,id3502065,1,2016-03-15 21:03:44,2016-03-15 21:14:37,1,-73.974014,40.757153,-73.949738,40.777092,N,653,2016-03-15,12,59,42445926,42448254
4,4,374,id3817493,2,2016-03-14 14:57:56,2016-03-14 15:15:26,1,-73.952881,40.766468,-73.978630,40.761921,N,1050,2016-03-14,11,29,42429694,42440012


In [26]:
graph = graph_from_routes.copy()

In [27]:
for i, node in enumerate(graph.nodes()):
    graph.nodes[node]['id'] = i

In [28]:
def distance_matrix_from_graph(G):
    """
    Create a distance matrix from a networkx graph.
    """
    num_nodes = len(G.nodes)
    distance_matrix = [[0] * num_nodes for _ in range(num_nodes)]
    for u in range(num_nodes):
        for v in range(num_nodes):
            if u != v:
                distance_matrix[u][v] = nx.shortest_path_length(G, source=u, target=v, weight='weight')
    return distance_matrix



In [29]:
class Vehicle():
    def __init__(self, id, pcapacity=4,gcapacity = 10, 
                 discharge_rate = 1.5, 
                 start_depot = 0, 
                 end_depot = 0 ):
        self.id = id
        self.pcapacity_ = pcapacity
        self.pcapacity_ = gcapacity
        self.r_ = discharge_rate
        self.charge = 100
        self.start_depot = start_depot
        self.end_depot = end_depot
        self.requests_ ={}
    def update_requests(self, id, start, dest):
        self.requests_[id] = (start,dest)
    def update_requests(self, request):
        self.requests_[request.id] = (request.start,request.dest)

    def update_charge(self,distance, speed = 50):
        self.charge -=  distance/speed * self.r_


In [30]:
from enum import Enum
class Charger(Enum):
    SLOW = (1,0.6)
    MEDIUM = (1,0.8)
    FAST = (2,0.8)

class Depot():
    def __init__(self, node_id, type = Charger.SLOW, vcapacity = 50, vehicles_now = 25) -> None:
        self.id = node_id
        self.r, self.omega = type.value
        self.vcapacity_ = vcapacity
        self.vehicles_now = vehicles_now


In [31]:
class Request():
    def __init__(self, id, people, start, dest) -> None:
        self.id = id
        self.people_ = people
        self.start = start
        self.dest = dest 


In [32]:
def convert_reqs_to_obj(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    reqs_obj = {}
    for node, item in requests.items():
        amount = item["amount"]
        temp_reqs = []
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
        reqs_obj[node] = temp_reqs
    
    return reqs_obj

In [143]:
def convert_reqs_to_list(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    temp_reqs = []
    for node, item in requests.items():
        amount = item["amount"]
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
    print(len(temp_reqs))
    return temp_reqs

In [144]:
def get_requests(dataframe, required_info = ["amount","internal_ids", "id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    requests = {}
    pick_up_nodes = np.unique(dataframe["pickup_graph_node"])
    
    for node in pick_up_nodes:
        temp_dict = dataframe[dataframe["pickup_graph_node"]==node].to_dict()
        temp_dict["amount"] = len(temp_dict["id"])
        temp_dict["internal_ids"] = list(temp_dict["id"].keys())
        requests[node] = {x:temp_dict[x] for x in required_info}
    reqs_obj = convert_reqs_to_obj(requests)
    reqs_list = convert_reqs_to_list(requests)
    return requests, reqs_obj, reqs_list

In [145]:
def initialize_depots(graph, vehicles_amount = [1]*5,type = [Charger.FAST]*5 ):
    depots = []
    idx = 0
    for node_id, data in graph.nodes(data = True):
        if data["depot"]:
            depots += [Depot(node_id, type = type[idx], vehicles_now = vehicles_amount[idx])]
            idx += 1
    return depots

In [146]:
def initialize_vehicles_naive(depots):
    vehicles = []
    #Calculate the amount of vehicles at the beginning
    n_vehicles = sum(d.vehicles_now for d in depots)
    for i,d in enumerate(depots):
        vehicles+= [Vehicle(id = i ,
                            pcapacity=4, 
                            gcapacity=10, 
                            discharge_rate=1,
                            start_depot= d.id,
                             end_depot= d.id) for i in range(d.vehicles_now*(i), d.vehicles_now * (i+1))]
    return vehicles, n_vehicles

In [147]:
depots = initialize_depots(graph)
vehicles, n_vehicles = initialize_vehicles_naive(depots)

In [148]:
_, _, requests = get_requests(df[:10])
print(requests)

10
[<__main__.Request object at 0x7f78113b2c40>, <__main__.Request object at 0x7f78113b2f10>, <__main__.Request object at 0x7f78113b2d90>, <__main__.Request object at 0x7f78113b2e80>, <__main__.Request object at 0x7f78113b2ca0>, <__main__.Request object at 0x7f78113b2f40>, <__main__.Request object at 0x7f78113b2be0>, <__main__.Request object at 0x7f78113b2c10>, <__main__.Request object at 0x7f781139eac0>, <__main__.Request object at 0x7f781139ec70>]


In [134]:
print(len(graph.edges())*n_vehicles)

1410


In [135]:
list(graph.nodes()).index(42447076)

22

In [136]:
def make_double_sense(G):
    for u in G.nodes():
        for v in G.nodes():
            # Check if the edge (u, v) does not already exist and if u != v
            if u != v and G.has_edge(u, v):
                # Add the edge (u, v) and (v, u)
                G.add_edge(v, u)
    return G

graph = make_double_sense(graph)

In [137]:
(42429374, 42436439) in graph.edges()

True

In [138]:
len(graph.edges())

282

In [160]:
def create_problem(graph,depots, vehicles, requests, name="VehicleRoutingProblem"):
    problem = pulp.LpProblem(name, pulp.LpMinimize)

    #Routing variable
    V = {}
    for a in vehicles:
        for u in graph.nodes():
            for v in graph.nodes():
                if graph.has_edge(u, v):
                    V[a.id,u, v]= pulp.LpVariable(f"V^{a.id}_{u}_{v}", cat='Binary') 
    
    #Request Assigning variable
    x = {}
    for a in vehicles:
        for r in requests:
            x[a.id,r.id]= pulp.LpVariable(f"x{a.id},{r.id}", cat='Binary') 



    # Define the objective function: minimize un assigned requests
    problem += pulp.lpSum(x[a.id,r.id] for a in vehicles for r in requests)



    #(4.5)
    for a in vehicles:
        problem += pulp.lpSum(x[a.id,r.id]*r.people_ for r in requests) <= a.pcapacity_

    #(4.16)
    for a in vehicles:
        temp_list = list(graph.nodes()).copy()
        temp_list.remove(a.start_depot)
        if a.start_depot != a.end_depot:
            temp_list.remove(a.end_depot)
        for v in temp_list:
            problem += pulp.lpSum(V[a.id,u, v] for u in temp_list if graph.has_edge(u, v)) \
                    - pulp.lpSum(V[a.id,v, w] for w in temp_list if graph.has_edge(v, w)) == 0
    
    #(4.17), #(4.18)
    for a in vehicles:
        problem += pulp.lpSum(V[a.id,a.start_depot, v] for v in temp_list if graph.has_edge(a.start_depot, v)) ==1
        if a.start_depot != a.end_depot:
            problem += pulp.lpSum(V[a.id,a.end_depot, v] for v in temp_list if graph.has_edge(a.end_depot, v)) ==1
    

    #(4.19), #(4.20)
    """       NO ASSIGNMENT 
    for a in vehicles:
        problem += pulp.lpSum(V[a.id,a.start_depot, v] for v in temp_list if graph.has_edge(a.start_depot, v)) ==1
        if a.start_depot != a.end_depot:
            problem += pulp.lpSum(V[a.id,a.end_depot, v] for v in temp_list if graph.has_edge(a.end_depot, v)) ==1
    """
    #(4.25), #(4.26)
    for r in requests:
        for a in vehicles:
            problem += pulp.lpSum(V[a.id,r.start, v] for v in temp_list if graph.has_edge(r.start, v)) ==x[a.id,r.id]
            if a.start_depot != a.end_depot:
                problem += pulp.lpSum(V[a.id,r.end, v] for v in temp_list if graph.has_edge(r.end, v)) ==x[a.id,r.id]
    
    return problem

In [162]:
problem = create_problem(graph,depots, vehicles, requests)

In [163]:
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/8c3784f3ca9c4403a88253d4d56d170a-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/8c3784f3ca9c4403a88253d4d56d170a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 435 COLUMNS
At line 12029 RHS
At line 12460 BOUNDS
At line 15272 ENDATA
Problem MODEL has 430 rows, 2811 columns and 5901 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 70 substitutions
Cgl0004I processed model has 400 rows, 2723 columns (2723 integer (2723 of which binary)) and 5628 elements
Cutoff increment increased from 1e-05 to 0.9999


1

In [164]:
print("Status:", pulp.LpStatus[problem.status])
print("unserved requests:", pulp.value(problem.objective))


Status: Optimal
Total Distance: 0.0
